<a href="https://colab.research.google.com/github/sander-ali/RAG-with-Gemini-and-Langchain/blob/main/RAG_with_Gemini_and_Langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Install necessary libraries
!pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf


In [ ]:
from IPython.display import display
from IPython.display import Markdown
import textwrap


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
import google.generativeai as genai
from google.colab import userdata

In [ ]:
#Get your API Key from google studio and store it in your secrets
import os
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Text Generation

In [ ]:
model = genai.GenerativeModel(model_name = "gemini-pro")
model

In [ ]:
response = model.generate_content("What are the usecases of LLMs?")

In [ ]:
to_markdown(response.text)

## Use LangChain to Access Gemini API

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY)

In [ ]:
result = llm.invoke("What are the usecases of LLMs?")

In [ ]:
to_markdown(result.content)

## Chat with Documents using RAG (Retreival Augment Generation)

In [ ]:
import PIL.Image

img = PIL.Image.open('/content/langchain.jpg')
img

![image.png](https://github.com/sander-ali/RAG-with-Gemini-and-Langchain/blob/main/langchain.jpg?raw=true)

In [ ]:
#Install dependencies
!sudo apt -y -qq install tesseract-ocr libtesseract-dev

!sudo apt-get -y -qq install poppler-utils libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

!pip install langchain

In [ ]:
#import necessary packages
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA



warnings.filterwarnings("ignore")
# restart python kernal if issues with langchain import.

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY,
                             temperature=0.2,convert_system_message_to_human=True)

## Extract text from the PDF

In [ ]:
# I am using a very recent paper titled "MemoRAG: Moving towards Next-Gen RAG Via Memory-Inspired Knowledge Discovery"
pdf_loader = PyPDFLoader("/content/2409.05591v2.pdf")
pages = pdf_loader.load_and_split()
print(pages[2].page_content)

In [ ]:
len(pages)

## RAG Pipeline: Embedding + Gemini (LLM)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)

In [ ]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)

In [ ]:
vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})

In [ ]:
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True
)

In [ ]:
question = "What is the difference between Standard RAG and MemoRAG?"
result = qa_chain({"query": question})
result["result"]

In [ ]:
Markdown(result["result"])

In [ ]:
result["source_documents"]

In [ ]:
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question = "What is the difference between Standard RAG and MemoRAG?"
result = qa_chain({"query": question})
result["result"]

In [ ]:
Markdown(result["result"])

In [ ]:
question = "Describe Quasar and Black Hole?"
result = qa_chain({"query": question})
Markdown(result["result"])